<a href="https://colab.research.google.com/github/jaworskamalgorzata99-GIT/Maching-Learning-Bootcamp/blob/main/unsupervised/association_rules/02_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import bibliotek

In [1]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

###Załadowanie danych

In [3]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2025-10-06 15:55:06--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 142.251.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv’

products.csv        100%[===================>]   2.07M  1.97MB/s    in 1.0s    

2025-10-06 15:55:08 (1.97 MB/s) - ‘products.csv’ saved [2166953/2166953]

--2025-10-06 15:55:08--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 142.251.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet-stre

In [4]:
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [5]:
orders = pd.read_csv('orders.csv', usecols=['order_id','product_id'])
orders

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633
...,...,...
1384612,3421063,14233
1384613,3421063,35548
1384614,3421070,35951
1384615,3421070,16953


###Przygotowanie danych

In [6]:
data = pd.merge(orders, products, how='inner', on='product_id', sort=True)
data = data.sort_values(by='order_id')
data.head()

,order_id,product_id,product_name
325660,1,13176,Bag of Organic Bananas
1382192,1,49683,Cucumber Kirby
1375139,1,49302,Bulgarian Yogurt
1302365,1,47209,Organic Hass Avocado
588447,1,22035,Organic Whole String Cheese


In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
order_id,1384617.00,1706297.62,989732.65,1.00,843370.00,1701880.00,2568023.00,3421070.00
product_id,1384617.00,25556.24,14121.27,1.00,13380.00,25298.00,37940.00,49688.00


In [9]:
# rozkład produktów
data['product_name'].value_counts()

,count
product_name,
Banana,18726
Bag of Organic Bananas,15480
Organic Strawberries,10894
Organic Baby Spinach,9784
Large Lemon,8135
...,...
Radiant Infinity Overnight Light Clean Scent Pads With Wings,1
Classic Original Lip Balm SPF 12,1
Sweet & Thick Original BBQ Sauce,1


In [10]:
# liczba transakcji

In [11]:
data['order_id'].nunique()

131209

In [15]:
transactions = data.groupby(by='order_id')['product_name'].apply(lambda name: ','.join(name))
transactions

,product_name
order_id,
1,"Bag of Organic Bananas,Cucumber Kirby,Bulgaria..."
36,"Grated Pecorino Romano Cheese,Organic Garnet S..."
38,"Green Peas,Bunched Cilantro,Flat Parsley, Bunc..."
96,"Organic Pomegranate Kernels,Organic Blueberrie..."
98,"Organic Ketchup,Queso Fresco,Aluminum Foil,Org..."
...,...
3421049,"Organic Baby Broccoli,Organic Whole Grain Whea..."
3421056,"Tartar Sauce,Homestyle Classics Meatloaf,Spark..."
3421058,"Club Soda Lower Sodium,Classic Britannia Crisp..."


In [16]:
# przekształcamy sobie na listę
transactions = transactions.str.split(',')
transactions

,product_name
order_id,
1,"[Bag of Organic Bananas, Cucumber Kirby, Bulga..."
36,"[Grated Pecorino Romano Cheese, Organic Garnet..."
38,"[Green Peas, Bunched Cilantro, Flat Parsley, ..."
96,"[Organic Pomegranate Kernels, Organic Blueberr..."
98,"[Organic Ketchup, Queso Fresco, Aluminum Foil,..."
...,...
3421049,"[Organic Baby Broccoli, Organic Whole Grain Wh..."
3421056,"[Tartar Sauce, Homestyle Classics Meatloaf, Sp..."
3421058,"[Club Soda Lower Sodium, Classic Britannia Cri..."


###Kodowanie transakcji

biblioteki (np. scipy.sparse, sklearn) pozwalają na używanie rzadkich struktur danych, które:

nie zapisują zer w pamięci,

przechowują tylko niezerowe wartości i ich pozycje,

zajmują dużo mniej RAM-u.

jeśli dasz sparse=False → dostaniesz zwykłą macierz NumPy (pełną, dense).

jeśli dasz sparse=True → dostajesz macierz rzadką (sparse),
czyli strukturę oszczędzającą pamięć (bo większość wartości to 0).

In [17]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
transaction_encoded = encoder.transform(transactions, sparse=True)
transaction_encoded

<Compressed Sparse Row sparse matrix of dtype 'bool'
	with 1442410 stored elements and shape (131209, 40434)>

In [19]:
transactions_encoded_df = pd.DataFrame(transaction_encoded.toarray(), columns=encoder.columns_)
transactions_encoded_df

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,...,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


###Algorytm Apriori

In [21]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.01, use_colnames=True)
supports = supports.sort_values(by='support', ascending=False)
supports.head(10)

,support,itemsets
8,0.14,(Banana)
7,0.12,(Bag of Organic Bananas)
76,0.08,(Organic Strawberries)
41,0.07,(Organic Baby Spinach)
31,0.06,(Large Lemon)
37,0.06,(Organic Avocado)
61,0.06,(Organic Hass Avocado)
100,0.05,(Strawberries)
33,0.05,(Limes)
69,0.04,(Organic Raspberries)


support (wsparcie) — jak często oba produkty występują razem,

confidence (pewność) — z jakim prawdopodobieństwem masło pojawia się, gdy jest chleb,

lift (podniesienie) — jak bardzo obecność chleba zwiększa szansę zakupu masła.

In [22]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:,[0,1,4,5,6]]
rules = rules.sort_values(by='lift', ascending=False)
rules.head(15)

,antecedents,consequents,support,confidence,lift
27,( Bag),(Clementines),0.01,0.79,36.84
26,(Clementines),( Bag),0.01,0.52,36.84
22,(Limes),(Large Lemon),0.01,0.26,4.26
23,(Large Lemon),(Limes),0.01,0.20,4.26
19,(Organic Raspberries),(Organic Strawberries),0.01,0.30,3.63
18,(Organic Strawberries),(Organic Raspberries),0.01,0.15,3.63
31,(Organic Avocado),(Large Lemon),0.01,0.18,2.94
30,(Large Lemon),(Organic Avocado),0.01,0.17,2.94
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.02,0.33,2.81
3,(Bag of Organic Bananas),(Organic Hass Avocado),0.02,0.16,2.81
